In [1]:

from __future__ import print_function
import torch, h5py
import numpy as np
from scipy.io import loadmat
from torch.nn.utils import weight_norm

import torch.nn as nn
import torch.optim as optim
import numpy as np

from torch.autograd import Variable

import torch.nn.functional as F
import customized_trmf_class as ctf

import skopt
# !pip install scikit-optimize if  necessary
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer  


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Predict future

In [2]:
np.random.seed(111)
torch.cuda.manual_seed(111)
torch.manual_seed(111)

import pandas as pd

# Ymat = np.load("/Users/hzhu/Downloads/Data/electricity.npy")

df = pd.read_csv("/Users/hzhu/Downloads/Data/postal_avails.csv",sep='\t')
width = len(df.flight_date.unique())
height = len(df.postal_code.unique())
Ymat = df.avails.values.reshape(width,height).T

In [3]:

rank = 64
size = 70
leads = 14
alt_iter = 2
epochs_factors = 1
epochs_tau = 1
lr_factors = 0.001
lr_tau = 0.001
reg_factors = 0.001
batch = 10
model = 'Net'


trmf = ctf.CusomizedTRMF(Ymat,rank,size,leads,alt_iter,epochs_factors,epochs_tau,lr_factors,lr_tau,reg_factors,batch,model)
F,X,model,l_F,l_X,l_X_temporal,l_tau = trmf.train()
predictions = trmf.predict_futures()
predictions = trmf.transfer2original(predictions.detach().numpy())
test = trmf.transfer2original(trmf.test)
median,mape_matrix = trmf.evaluation(predictions,test)
median



training factors
training tau model


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


0.18996917429066482

# impute missing

In [4]:

df = pd.read_csv("/Users/hzhu/Downloads/Data/postal_avails.csv",sep='\t')
width = len(df.flight_date.unique())
height = len(df.postal_code.unique())
Ymat = df.avails.values.reshape(width,height).T

In [5]:

rank = 64
size = 70
leads = 0
alt_iter = 2
epochs_factors = 1
epochs_tau = 1
lr_factors = 0.001
lr_tau = 0.001
reg_factors = 0.001
batch = 10
model = 'Net'

trmf = ctf.CusomizedTRMF(Ymat,rank,size,leads,alt_iter,epochs_factors,epochs_tau,lr_factors,lr_tau,reg_factors,batch,model)
F,X,model,l_F,l_X,l_X_temporal,l_tau = trmf.train()


training factors
training tau model


In [6]:
data = trmf.impute_missings()

In [7]:
data

array([[  279.99994096,    75.99999736,   172.99998197, ...,
          508.99995001,   522.00002894,   368.99996333],
       [  179.00000042,   416.99994565,   421.99996296, ...,
          350.9999309 ,    94.00000302,   132.99999554],
       [42888.9989549 , 45222.99975282, 40147.9993104 , ...,
        47514.99826977, 56508.99852978, 42691.00013903],
       ...,
       [  310.99992939,   209.00002619,   401.99995969, ...,
          459.99995377,   322.99994257,   261.9999452 ],
       [ 1863.99998923,  1642.00000196,  1780.00005314, ...,
         1129.99997503,  1681.99986741,  2135.00006937],
       [ 2931.99994028,  3672.99960811,  2840.99965363, ...,
         2622.9996671 ,  3173.00003462,  2942.00009908]])

# Bayesian Optimization

In [75]:
rank = Integer(low=50, high=500, name='rank')
size = Integer(low=7, high=210, name='size')
alt_iter = Integer(low=2, high=50, name='alt_iter')
epochs_factors = Integer(low=1, high=10, name='epochs_factors')
epochs_tau = Integer(low=1, high=10, name='epochs_tau')
lr_factors = Real(low=1e-4,high=1e-1,name="lr_factors")
lr_tau = Real(low=1e-4,high=1e-1,name="lr_tau")
reg_factors = Real(low=1e-4,high=1e-1,name="reg_factors")

dimensions = [
    rank,
    size,
    alt_iter,
    epochs_factors,
    epochs_tau,
    lr_factors,
    lr_tau,
    reg_factors,
]

In [76]:
def fitness(dimensions):
#     Ymat = np.load("/Users/hzhu/MyCode/DeepGLO/datasets/electricity.npy")
    df = pd.read_csv("/Users/hzhu/Downloads/Data/postal_avails.csv",sep='\t')
    
    width = len(df.flight_date.unique())
    height = len(df.postal_code.unique())
    Ymat = df.avails.values.reshape(width,height).T
    
    trmf = ctf.CusomizedTRMF(Ymat,dimensions[0],dimensions[1],14,dimensions[2],dimensions[3],
                             dimensions[4],dimensions[5],dimensions[6],dimensions[7],10,'Net')
    F,X,model,l_F,l_X,l_X_temporal,l_tau = trmf.train()
    predictions = trmf.predict_futures()
    predictions = trmf.transfer2original(predictions.detach().numpy())
    test = trmf.transfer2original(trmf.test)
    median,mape_matrix = trmf.evaluation(predictions,test)
    
    return round(median,4)
    
default_parameters = [64,70,2,1,1,0.001,0.001,0.001]
gp_result = gp_minimize(func=fitness,dimensions=dimensions,verbose=True,n_jobs=4,n_calls=30,x0=default_parameters)
params = []
params.append(gp_result.x)
# a = gp_result.x
# with open("param.txt", "w") as f:
#     for s in a:
#         f.write(str(s) +"\n")

Iteration No: 1 started. Evaluating function at provided point.
training factors
training tau model
Iteration No: 1 ended. Evaluation done at provided point.
Time taken: 2.2613
Function value obtained: 0.2049
Current minimum: 0.2049
Iteration No: 2 started. Evaluating function at random point.
training factors
training tau model
training factors


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([151, 1])) that is different to the input size (torch.Size([151])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 16.4873
Function value obtained: 96.9214
Current minimum: 0.2049
Iteration No: 3 started. Evaluating function at random point.
training factors
training tau model
training factors


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([387, 1])) that is different to the input size (torch.Size([387])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 27.4094
Function value obtained: 152.2625
Current minimum: 0.2049
Iteration No: 4 started. Evaluating function at random point.
training factors
training tau model
training factors


...

training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
Iteration No: 28 ended. Search finished for the next optimal point.
Time taken: 12.3250
Function value obtained: 4.7653
Current minimum: 0.2049
Iteration No: 29 started. Searching for the next optimal point.
training factors
training tau model


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([282, 1])) that is different to the input size (torch.Size([282])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
training factors
Iteration No: 29 ended. Search finished for the next optimal point.
Time taken: 38.6371
Function value obtained: 0.2772
Current minimum: 0.2049
Iteration No: 30 started. Searching for the next optimal point.
training factors
training tau model
training factors


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([317, 1])) that is different to the input size (torch.Size([317])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


training tau model
training factors
training tau model
training factors
training tau model
training factors
training tau model
Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 13.6038
Function value obtained: 387.6365
Current minimum: 0.2049
